### Word Representation

Previously we were representing words with on-hot vector using vocabluary.

Weackness is that the word is treated as independent thing. Hurts generalization. No relationship between words of similar meaning is here.

Intesd it is better to learned `feauturized represenation`. Fow words, features like age, royalty, gender... 

Each feature vector in this represenation is denoted as $e_{N}$, where $N$ is the number of features for a given work represenation. Similarity in feature vectors between words would allow to better _generalized_ for algorithm.  

It is also common to embede the _visualize word embedding_ using `t-SNE` algorithm.  High dimension embedding visualized in 2D space.  

The featureized represenations are called `embeddings`.  

### Using word embeddings

Embeddings help generalizing.  

Examining large amounts of unlabeled text allows to learn embeddings that allow to group similar words together.  

This is done via __transfer learning__. Using generalized learned represenations to do tasks with _small train_ set. 

_The exact algorithm is the mollowing_:  

- Learn word embeddings from large text corpus.  (use pre-trained embeddings)
- Transfer embeddings for a task with small train set (lower dimension feature wectors)
- Continue to finetune the word embedding with new data

This is less usefull for tasks where there are a lot of train data, e.g., machine translation. 


Word embeddings have similarity with face recognition in CNNs.  The difference is that in CNNs the unseen picture can be used, while in word embeddings, words should have known feature vector. 

> Encoding $\approx$ Embedding

### Properties of word embeddings

This helps in analogy reasonigs.  
#e_1 - e_2 \approx e_{i}-?# the algorithm is trying to find the $?$, -- which embeddings are as similar as those that are asked. Man - Woman, King - ? (Queen). 

Each word in the _embedding space_ is represented as a point. This is a high dimensional space. Goal is find a point that _maximises the similarity_ between a required $e_1-e_2+e_{i}$ and $e_{?}$  where $e_{?}$ is the one to be found.  

Visualization for embedding psaces, aka, t-SNE, performs a non-linear mapping from $N-D$ embedding space to 2D planne. _After mapping this similarity no longer holds_. 

The most commonly used is `cosine similarity` as 
$$
sim(u,v) = \frac{u^T v}{||u||_2 ||v||_2}
$$
-- cosine of the inner product between two vectors; 
which is just an inner product.  

Also `eucleadian distance` is used 
$$
||u-v||^2
$$

the main difference is the normalization for the length of vectors.  

### Embedding Matrix

Consider a $V$-word wocabluary.  
Learn an embedding matrix $E$ that is $F$-features by $V$-vocabluary size matrix.  

Multiplying the embedding matrix by the one-hot vector gives the vector with the higher of the feature vector $F$.  

This multiplication selects a specific word, by zeroes out everything else.  

### Learning word Embeddings. 

There are many algorithms for word embeddings.  

Consider a sesntence and a NN to complete it. Each word has an index in the vocab.  
For each word, we constract a _one-hot_ vector.  
Constract the matrix of arameters $E$, and 
$$
o_{i} \rightarrow E \rightarrow e_{i}
$$
And the same for all remaining vectors.  

- Now we have a set of features-size-$D$ embedding vectors.  
- Feed them into a NN with a hidden lyaer, with set of parameters $W^{[1]}$ $b^{[1]}$ with the size of $N$ of words $\times$  size of the feature vector. Otherwise `fixed historical window` is used to limit how long is the sequence of words that is used for prediction (`context window?`) 
- Feed the result into _soft-max_  $W^{[2]}$ $b^{[2]}$.  
that classifies the output from the vocab.  

Here the _same_ matrix $E$ is used for _all_ words.  

The parameters, weights and biases, are optimized using the _backprop_. 

This algorithm words well. But there are fancier/or/simpler versions.  


#### Other context/target pairs

`Target word` is the one to predict.  
`Context` are words that go before it.  

We can learn by settings context both in the _left_ and on the _right_ from the target. 

Or consider a nearby _one_ word, which is a `skip gram` model.  

This simpler approach allows to learn _word embedding_.  

### Word2Vec model

THis is simpler an computationally more efficinet way.  

In the classical _skip-gram_ model, we consider pairs "context-target" to perform supervised learning.  

Randomly pick a word for context and randomly pick the target and form pairs this way.  
Now, isntead of supervised learning, consider which of the +/- 10 words from it in the context is a good target.  

##### Word2Vec: Skip-Gram Model

Consider a vocab of $V$ words.  
The supervised problem we have is to learn mapping from context $X\rightarrow Y$ target.   
Create a _one-hot_ vector $o_i$m take the embedding matrix $E$ and infer the embedding $e_c$. This gives an embedding for a context word.  
Than use the $e_c$ feed it into a softmax and it returns the _probability_ of different contect words 

$$
p(t,c) = \frac{e^{\theta_t^T}e_c}{\sum_{j=1}^V e^{\theta_j^T}e_c}
$$

where $\theta_t$ is the parameter $\sim$ output $t$ (chance of the word $t$ to be the label). 

Loss function for the _soft-max_ is __negative log likelihood__:  

$$
\mathcal{L}(\hat{y},y) = -\sum_{i=1}y_i\log\hat{y}_i
$$
 
with $y$ being a one-hot vector.  
and $\hat{y}$ is the vector of $V$ probabilities for each word in the vocab. to be a good label.  

This is a _little NN_ 

$$
o_c \rightarrow E \rightarrow e_c \rightarrow [softmax] \rightarrow \hat{y}
$$

that has parameters in embedding $E$ (for each embedding vector $e_c$) and softmax with parameters $\theta_c$.  
If we optimize this mddel, we arrive at good parameters.  

__Problems__:  Computational speed. _softmax is expensive to compute_ The probability evaluation requires the sum of all $V$ entries in the vocab.  

__SOlution__: `hierachical soft-max`. (classifier splits the vocab into sections and first finds the section and then the word, like a _tree of clasifiers_). The computational cost here scales as $\log(V)$ rather than linear. Where the trees are not perfectly balanced.  

Sampling the context $c$. The context is chosen either by uniformly randomly, but this leads to words like 'the' 'a' are found extrmeely frequently. This biases the updates.  The sampling are done with a certain _heirsitcs_ to account for common/uncommon words. 



#### Negative sampling

Sample context and target words.  
And generate positive examples (as before, within the context window). 
Generate negative examples by randomly picking the word from the dictionary.  Most likely it should not be associated. Than the labels would $0$.  

Given the pair or words, what is the probability of them appearing together?

So the training set consists of one positive, $y=1$ example (from context window) and $k$ negative $y=0$ randomly chosen from the vocab. $k$ is _small_ for large dataset and _large_ for smaller datasets.  

- With K to 1 ration of examples.

Than create a supervised NN.  

Again, define a logistic regression model, a sigmoid

$$
P(y=1|c,t) = \sigma(\theta_{c}^T,e_c)
$$

$$
o_{n+p} \rightarrow E \rightarrow [... binary\: classifiers ...]
$$

Here, instead of giant soft-max, we consider giant number of binary classifications, that are __much cheaper__. 

How to chose negative examples? 
- Random sampling of words 
- Sampling according with  empiricalfrequency (of the word appearance )
- Middle point $P(w_i) = f(\omega_i)^{3/4} / \sum(f(w_j)^{3/4})$






### GloVe algorithm - global vectors for word represenation

A very simple algorithm.  

We sample words that appear in a close proximity, pairs $c,t$ as  
$X_{ij}$ = \# of times $j$ appears in context of $i$,  
whre $j=t$ and $i=c$.  
$X_{i,j}$ counrs how ofter a given pair appears.  
The algorithm is to

$$
\min \sum_{i=1}\sum_{j=1} f(X_{ij})(\theta^T_j e_j + b_i + b_j' - \log(X_{ij}))^2
$$

and solve fot parameters $\theta$ and $e$, to minimize. 

where again $i=t$ and $j=c$, $f(X_{ij})$ is the weighting term, that is $0$ if $i=j$ to avoid the problem with log.   
THis equations says how related the words, $c,t$ are.  
Weighting factor also allows to contol for heuristics (frequency of the word appearance).  

Here the rols of $\theta$ and $e$ are completely symmeric. So they can be 
- initialized with random numbers
- aget training compute $e_{w}^{final} = (e_w + \theta_w)/2$.  

**Notably** individaul components of the embedding are not generally interpretable. 


### Sentiment classification

Determination of the tone/emotion/number behind the text. 

Generally, not much training set is available for it.  

Word embeddings can help to do with this task.  

Approaches:  
- Use a classifier, one-hot vector, use emedding matrix from other task, extract embedding vector, and this gives encodings for new words using much larger dataset.  
Then, average the embedded vector that can be passed into soft-max classifier with, e.g., 1-5 outputs. This allows for an argitrary sized text to extract one number/numbers to assess the setiment.  

Negative: this approach ignores word order. 

Another approach is to use RNN for sentiment classification. Again, it starts with one-hot vector, that is fed into RNN, that allows to predict $\hat{y}$ via _many-to-one_ RNN achitecture. 

### Bias in a word embeddings

Ah, there is no bias :) 

- finihsed exercises